
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2022**

---

# Trabajo práctico entregable - Parte 2

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn
import math
import scipy as sp
seaborn.set_context('talk')

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [2]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [3]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,...,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,...,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,...,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,...,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN


In [4]:
df[['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                                     \
                             count          mean            std   min   
profile_gender                                                          
Hombre                      4944.0  98836.063558  102159.664237   1.0   
Mujer                        908.0  73501.414465   56256.880181   2.0   
Otros                         31.0  95441.354839   88635.674358  40.0   

                                                       
                    25%      50%       75%        max  
profile_gender                                         
Hombre          53285.0  80000.0  110000.0  2080000.0  
Mujer           43818.0  65000.0   92000.0   800000.0  
Otros           39500.0  74000.0  117500.0   380000.0

In [5]:
df[df.salary_monthly_NETO > 1000]\
  [['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                                         \
                             count           mean            std      min   
profile_gender                                                              
Hombre                      4815.0  101481.624343  102215.304684   1100.0   
Mujer                        863.0   77328.913778   55082.760570   2000.0   
Otros                         28.0  105660.714286   87235.710235  34000.0   

                                                       
                    25%      50%       75%        max  
profile_gender                                         
Hombre          55000.0  80000.0  110000.0  2080000.0  
Mujer           47850.0  67200.0   93000.0   800000.0  
Otros           43750.0  78900.0  122750.0   380000.0

In [6]:
alpha = 0.05

In [7]:
is_man = df.profile_gender == 'Hombre'

groupA = df[(df.salary_monthly_NETO > 1000) & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man].salary_monthly_NETO #~ es la negacion del filtro, o sea los generos distintos a 'hombre'

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

In [8]:
media_A = groupA.mean()
media_B = groupB.mean()
std_A = groupA.std()
std_B = groupB.std()
N_a = len(groupA)
N_b = len(groupB)
error_std_A = std_A/math.sqrt(N_a)
error_std_B = std_B/math.sqrt(N_b)
print('La media del grupo de varones es ', media_A)
print('La media del grupo de otros generos es ', media_B)
print('La desviación estandar del grupo de varones es ', std_A)
print('La desviación estandar del grupo de otros generos es ', std_B)
print('El número de muestras del grupo de varones es ', N_a)
print('El número de muestras del grupo de otros generos es ', N_b)
print('El error estandar del grupo de varones es ', error_std_A)
print('El error estandar del grupo de otros generos es ', error_std_B)

La media del grupo de varones es  101481.62434267918
La media del grupo de otros generos es  78219.25094276095
La desviación estandar del grupo de varones es  102215.30468444877
La desviación estandar del grupo de otros generos es  56515.32732350074
El número de muestras del grupo de varones es  4815
El número de muestras del grupo de otros generos es  891
El error estandar del grupo de varones es  1473.0509950457006
El error estandar del grupo de otros generos es  1893.3347033956811


Tenemos dos grupos con distintas desviaciones estandar, por lo que usamos el test de Welch 

In [9]:
est_dif_medias = media_A - media_B
t = (media_A-media_B)/math.sqrt(error_std_A**2+error_std_B**2)
nu = ((std_A**2/N_a + std_B**2/N_b)**2)/(std_A**4/(N_a**2*(N_a-1))+std_B**4/(N_b**2*(N_b-1)))
print('El estimador para la diferencia de medias es', est_dif_medias)
print('El estadistico de prueba para el test de Welch es ', t)
print ('El numero de grados de libertad de t-student para el test de Welch es', nu)

El estimador para la diferencia de medias es 23262.37339991823
El estadistico de prueba para el test de Welch es  9.697205976350695
El numero de grados de libertad de t-student para el test de Welch es 2148.0505828160885


Con un numero de grados de libertad tan grande, la distribución de t-student ya es una normal, por lo que usamos este estadistico para una distribución normal. Ademas, incluso si quisiera utilizar t.student, no podria (al menos de esta forma "manual" en la que estoy calculando), ya que no hay valores de tabla para los $t_{\nu,\alpha}$ para valores tan grandes de $\nu$. El valor de $t_{\frac{\alpha}{2},\nu-1}\sim z_{\frac{\alpha}{2}}$ para los valores $\alpha=0.05$, $\nu\sim2148$ esta dado por $z_{\frac{\alpha}{2}} = 1.96$, por lo que el intervalo de confianza estará dado por 
$\bar{X}-\bar{Y}\pm1.96\sqrt(s_{\bar{X_1}^2+s_{\bar{X_2}}^2)$

In [10]:
IC_min = est_dif_medias - 1.96*math.sqrt(error_std_A**2+error_std_B**2)
IC_max = est_dif_medias + 1.96*math.sqrt(error_std_A**2+error_std_B**2)
print('El intervalo de confianza 1-alpha es (',IC_min,',',IC_max,')')

El intervalo de confianza 1-alpha es ( 18560.5807418027 , 27964.16605803376 )


Este intervalo de confianza nos indica que de acuerdo a nuestra muestra, si la diferencia entre las medias de las distribuciones de salario es cero, entonces el estadístico de prueba caera dentro de este intervalo de confianza

Voy a repetir los resultados que aca calcule manualmente utilizando scipy.stats para hacer el test de Welch.

In [11]:
sp.stats.ttest_ind(groupA, groupB, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='greater', trim=0)

Ttest_indResult(statistic=9.697205976350673, pvalue=4.302598661066604e-22)

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos componentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0=...$

**Estadístico (Pivote)**
  * Identificar el estadístico
  * Escribir qué distribución tiene bajo $H_0$


Nuestra hipótesis nula sera que la diferencia de medias entre ambos grupos es cero, es decir $H_0:\mu_A-\mu_B=0$, y la hipótesis alternativa sera que $H_a:\mu_A-\mu_B>0$. Nuestro estadístico de prueba (pivote) estará dado por $$T=\frac{\bar{A}-\bar{Y}}{sqrt{s_{\bar{X}^2+s_{\bar{Y}^2}}}$$, que tiene distribución normal con media $\mu=0$ y desviación estandar $\sigma^2=1$ bajo $H_0$ (aca tengo dudaassss)

### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

El estadístico de prueba, calculado arriba, dio un valor de 9.697. No entendi que es el p-valor, pero otra forma de rechazar o no la hipótesis nula es calculando la region de rechazo para nuestra hipótesis. En nuestro caso, si nuestra hipotesis alternativa es la arriba mencionada, la región de rechazo estará dada por $RR=\{t\geq\:z_{\alpha}\}$. Buscando valores de tabla, $z_{\alpha} = 1.64$, por lo que nuestra región de rechazo será $RR=\{t\geq\:1.64\}$. Dado que el estadístico de prueba t es mayor que 1.96, cae en la región de rechazo, por lo que se favorece la hipótesis alternativa y se rechaza la hipótesis nula. Podemos concluir entonces que, de acuerdo a nuestra muestra, $\mu_A>\mu_B$, o sea que la media de los salarios de los varones es mayor que la media de los salarios para otros géneros. 

Otra forma de hacerlo, esta vez utilizando el p-valor que no supe como calcular, es utilizando scipy, que arroja el p-valor. Para un nivel de confianza $\alpha$, el p-valor se define como la probabilidad de obtener resultados al menos tan improbables o "extremos" como los observados en la muestra que tenemos, suponiendo que la hipótesis nula sea verdadera. Es decir, si tenemos una muestra de datos con un p-valor de 0,1, eso quiere decir que la probabilidad de haber obtenido los datos que tuvimos, siendo verdadera la hipótesis nula, es del 10%. Si anteriormente nos habiamos fijado una tolerancia o confianza de $\alpha=0.05$ o sea una tolerancia del 5%, eso quiere decir que nuestros datos son consistentes con la hipótesis nula y que esta no debe rechazarse en favor de la alternativa. En cambio, si por ejemplo tenemos un p-valor de 0.01, o sea un 1% de probabilidades de haber obtenido nuestros dados, debemos rechazar la hipótesis nula, ya que el p-valor esta por debajo de nuestra tolerancia. En resumen, si tenemos un nivel de confianza $\alpha$, la hipótesis nula no debe rechazarse si $\alpha$ es menor al p-valor; si por el contrario se da que $\alpha$ es mayor al p-valor, la hipótesis nula debe rechazarse en favor de la hipótesis alternativa. 

Calculemos el p-valor utilizando scipy. 

In [12]:
sp.stats.ttest_ind(groupA, groupB, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='greater', trim=0)

Ttest_indResult(statistic=9.697205976350673, pvalue=4.302598661066604e-22)

Observamos que nuestro p valor es $4.3\times10^{-22}$, es decir si las medias de los salarios de varones y otros géneros fueran iguales, los resultados que se obtuvieron en la encuesta se hubiesen obtenido con una probabilidad del $4.3\times10^{-22}$ (increiblemente baja!). Como nuestro $\alpha=0.05$, $\alpha$ es mayor (por veinte ordenes de magnitud) al p-valor, con lo cual se rechaza la hipótesis nula en favor de la alternativa, que es que los salarios del grupo A (varones) tienen una media mas alta que los del grupo B (otros generos). (Observemos que le pedimos a scipy que nuestra hipótesis alternativa sea esa cuando le ponemos alternative='greater', asi no solo sabemos que ambas medias son diferentes sino que se cumple, como esperado, que la media de salarios para varones es mayor).

### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

In [13]:
from statsmodels.stats.power import tt_ind_solve_power

In [14]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)

In [15]:
tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=ratio)

NameError: name 'power' is not defined

## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje, en un archivo PDF.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
  1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
  2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
  3. Puntos clave: 
    1. Simpleza de los gráficos.
    2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
    2. Selección de UNA oración sobre la que se hace énfasis.
    3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
  2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
  2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
  3. Puntos clave:
    3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
    4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle. 
    5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
    2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
  1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
  2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
  3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
  3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
  4. Puntos clave:
    1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
  3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.
